# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

Simimilarly, It is an adapted project from Mr.Mccullum, and I primarily follow his idea, use yfiannce to create my own stuff.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [ ]:
import numpy as np
import pandas as pd
import math
import yfinance as yf

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
tickers=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers=tickers.to_list()
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

In [106]:
apple= yf.Ticker('AAPL')
print(apple.info['regularMarketOpen'])
print(apple.info)

#肯定是没问题的，所以问题在于这个要在一个cell里！

241.97
{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcast

In [107]:
apple_info=yf.Ticker('AAPL').info
I_WANT=apple_info['52WeekChange']
print(I_WANT)

0.30087543


In [108]:
two_stocks=yf.Tickers('AAPL MSFT')
two_stocks.tickers['AAPL'].info['52WeekChange']
#different from the following
two_stocks.tickers['AAPL'].info.get('52WeekChange')
# difference is that when the value is missing, 1 will raise an error, 2 will return a None


0.30087543

In [109]:
#prepare chunk
chunk_size=150
first_result=[]
for i in range(0,len(tickers),chunk_size):
    chunk_tickers=tickers[i:i+chunk_size]
    chunk_tickers_str=' '.join(chunk_tickers)
    print(chunk_tickers_str)
    #这里chunk就好了
    #接下来开始一次性request 150个
    chunk_stocks=yf.Tickers(chunk_tickers_str)
    for ticker in chunk_tickers:
        single_stock_info=chunk_stocks.tickers[ticker].info
        price=single_stock_info.get('regularMarketOpen')
        price_change=single_stock_info.get('52WeekChange')
        first_result.append(
            {
                'Ticker': ticker,
                'Price': price,
                '1 Year Price Change':price_change,
                'Number of Share to Buy':'N/A'
            }
        )
    


MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK.B BBY TECH BIIB BLK BX BK BA BKNG BWA BSX BMY AVGO BR BRO BF.B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW CE COR CNC CNP CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG
DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ERIE ESS EL EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GE GEHC GEV GEN GNRC GD GIS GM GPC GILD GPN GL GDDY GS HAL HIG HAS HCA DOC HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUBB HUM

In [110]:
second_result=pd.DataFrame(first_result)
third_result=second_result.dropna(subset=['Price'])
third_result=third_result.dropna(subset=['1 Year Price Change'])
final_result= third_result.reset_index(drop=True)
final_result

,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,MMM,132.03,0.466232,N/A
1,AOS,67.87,-0.151342,N/A
2,ABT,113.54,-0.008741,N/A
3,ABBV,179.20,0.088786,N/A
4,ACN,361.00,0.032027,N/A
...,...,...,...,...
494,XYL,115.16,0.031540,N/A
495,YUM,127.79,-0.005031,N/A
496,ZBRA,393.83,0.551730,N/A
497,ZBH,102.92,-0.163110,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [111]:
final_result.sort_values('1 Year Price Change', ascending = False, inplace = True)
final_result=final_result[0:50]
final_result.reset_index(drop=True, inplace=True) # if there isn't drop=True, then the new frame will have an extra index
final_result

,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,VST,162.970,3.247790,N/A
1,PLTR,68.140,3.168553,N/A
2,GEV,362.310,1.594846,N/A
3,NVDA,142.660,1.578473,N/A
4,TPL,1253.970,1.496128,N/A
5,UAL,99.860,1.286715,N/A
6,AXON,565.020,1.256967,N/A
7,TRGP,187.790,1.241021,N/A
8,CEG,255.495,1.206560,N/A
9,AVGO,229.660,1.115920,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

1


In [113]:
# 建立随时输入的一个函数
def portfolio_input():
    global portfolio_size
    portfolio_size=input("How much money you have?")

    try:
        float(portfolio_size)
    except ValueError:
        print("Enter some real shit!")
        portfolio_size=input("How much money you have?")

#试一下
portfolio_input()
print(portfolio_size)

#就算每个买多少
position_size= float(portfolio_size)/len(final_result)
print(position_size)

for i in final_result.index:
    final_result.loc[i,'Number of Share to Buy']=float(position_size)/final_result.loc[i,'Price']

final_result




0.02


,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,VST,162.970,3.247790,0.000123
1,PLTR,68.140,3.168553,0.000294
2,GEV,362.310,1.594846,0.000055
3,NVDA,142.660,1.578473,0.00014
4,TPL,1253.970,1.496128,0.000016
5,UAL,99.860,1.286715,0.0002
6,AXON,565.020,1.256967,0.000035
7,TRGP,187.790,1.241021,0.000107
8,CEG,255.495,1.206560,0.000078
9,AVGO,229.660,1.115920,0.000087


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

Indeed, we need a more sophisticated strategy to win the market

In [163]:
# 一个的摸索例子，建立函数
from datetime import datetime,timedelta
months=int(input("how many months?"))
print(datetime.today())
print(timedelta(days=30 * months))

end_date=datetime.today()
start_date=end_date-timedelta(days=30 * months)
print(start_date)
print(end_date)

#print(apple.history(start=start_date,end=end_date))
apple_history=apple.history(start=start_date,end=end_date)
old_price=apple_history['Close'].iloc[0]
new_price=apple_history['Close'].iloc[-1]
stock_return1=(new_price-old_price)/old_price
print(stock_return1)


2025-01-09 00:19:59.467032
360 days, 0:00:00
2024-01-15 00:19:59.467447
2025-01-09 00:19:59.467447
0.3216794123766783


/var/folders/lr/8z2w833j035bvfvx86lnmdfh0000gn/T/ipykernel_68204/2004119663.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  old_price=apple_history['Close'][0]
/var/folders/lr/8z2w833j035bvfvx86lnmdfh0000gn/T/ipykernel_68204/2004119663.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_price=apple_history['Close'][-1]


In [172]:
#一个的例子，建立函数
from datetime import datetime,timedelta 
# 这里就构想，第二个parameter是一个object
def stock_return(months, one_stock):
    #时间
    end_date=datetime.today()
    start_date=end_date-timedelta(days=30 * months)
    #找价格
    stock_history=one_stock.history(start=start_date,end=end_date)
    global old_price
    old_price=float(stock_history['Close'].iloc[0])
    new_price=float(stock_history['Close'].iloc[-1])
    #Calculate return
    Stock_Return=(new_price-old_price)/old_price
    return float(Stock_Return)

print(stock_return(12,yf.Ticker('AAPL')))
print(old_price)
#牛逼！

0.3281544319808786
182.73477172851562


In [ ]:
#建立函数,复制上面的
from datetime import datetime,timedelta 
def stock_return(months, one_stock):
    end_date=datetime.today()
    start_date=end_date-timedelta(days=30 * months)
    stock_history=one_stock.history(start=start_date,end=end_date)
    global old_price
    old_price=float(stock_history['Close'].iloc[0])
    new_price=float(stock_history['Close'].iloc[-1])
    Stock_Return=(new_price-old_price)/old_price
    return float(Stock_Return)

#prepare chunk
chunk_size=150
first_result=[]
for i in range(0,len(tickers),chunk_size):
    chunk_tickers=tickers[i:i+chunk_size]
    chunk_tickers_str=' '.join(chunk_tickers)
    print(chunk_tickers_str)
    #这里chunk就好了
    #接下来开始一次性request 150个
    chunk_stocks=yf.Tickers(chunk_tickers_str)
    for ticker in chunk_tickers:
        try:
            single_stock=chunk_stocks.tickers[ticker]
            price=old_price
            price_change_1_month=stock_return(1,single_stock)
            price_change_3_month=stock_return(3,single_stock)
            price_change_6_month=stock_return(6,single_stock)
            price_change_12_month=stock_return(12,single_stock)
            
            first_result.append(
                {
                    'Ticker': ticker,
                    'Price': price,
                    '1 Month Price Change':price_change_1_month,
                    '1 Month Return Percentile':"N/A",
                    '3 Months Price Change':price_change_3_month,
                    '3 Months Return Percentile':"N/A",
                    '6 Months Price Change':price_change_6_month,
                    '6 Months Return Percentile':"N/A",
                    '12 Months Price Change':price_change_12_month,
                    '12 Months Return Percentile':"N/A",
                    'HQM(high-quality momentum score)':'N/A',
                    'Number of Share to Buy':'N/A'
                }
            )
        except Exception as e:
            print(f"{ticker} is wrong.")
            continue

# 2024-01-09跑成
        


In [191]:

print(len(first_result))
second_result=pd.DataFrame(first_result)
columns_list=second_result.columns.tolist()
third_result=second_result.dropna(subset=columns_list)
final_result= third_result.reset_index(drop=True)
print(len(final_result))
final_result


349
349


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,182.734772,0.036680,N/A,0.003677,N/A,0.323740,N/A,0.546590,N/A,N/A,N/A
1,AOS,86.984901,-0.058920,N/A,-0.146915,N/A,-0.211295,N/A,-0.138192,N/A,N/A,N/A
2,ABT,79.692963,-0.010823,N/A,-0.011467,N/A,0.114884,N/A,0.019425,N/A,N/A,N/A
3,ABBV,112.073029,0.016110,N/A,-0.073457,N/A,0.070793,N/A,0.134964,N/A,N/A,N/A
4,ACN,157.273666,-0.014246,N/A,-0.008344,N/A,0.129978,N/A,0.025160,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
344,NRG,17.250000,0.051318,N/A,0.102917,N/A,0.320161,N/A,0.979596,N/A,N/A,N/A
345,NUE,49.777840,-0.155210,N/A,-0.218329,N/A,-0.277182,N/A,-0.297880,N/A,N/A,N/A
346,NVDA,168.318756,0.037314,N/A,0.039463,N/A,0.091028,N/A,1.485717,N/A,N/A,N/A
347,NVR,56.366020,-0.096151,N/A,-0.160291,N/A,-0.007809,N/A,0.118035,N/A,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [198]:
Time_period_list=['1 Month','3 Months','6 Months','12 Months']
for i in Time_period_list:
    final_result[f"{i} Return Percentile"]=final_result[f"{i} Price Change"].rank(pct=True)

final_result

        
        


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,182.734772,0.036680,0.914040,0.003677,0.604585,0.323740,0.925501,0.546590,0.905444,N/A,N/A
1,AOS,86.984901,-0.058920,0.329513,-0.146915,0.160458,-0.211295,0.094556,-0.138192,0.146132,N/A,N/A
2,ABT,79.692963,-0.010823,0.736390,-0.011467,0.561605,0.114884,0.687679,0.019425,0.312321,N/A,N/A
3,ABBV,112.073029,0.016110,0.873926,-0.073457,0.372493,0.070793,0.575931,0.134964,0.504298,N/A,N/A
4,ACN,157.273666,-0.014246,0.704871,-0.008344,0.567335,0.129978,0.719198,0.025160,0.326648,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
344,NRG,17.250000,0.051318,0.945559,0.102917,0.851003,0.320161,0.916905,0.979596,0.982808,N/A,N/A
345,NUE,49.777840,-0.155210,0.022923,-0.218329,0.060172,-0.277182,0.045845,-0.297880,0.042980,N/A,N/A
346,NVDA,168.318756,0.037314,0.916905,0.039463,0.696275,0.091028,0.641834,1.485717,0.997135,N/A,N/A
347,NVR,56.366020,-0.096151,0.094556,-0.160291,0.128940,-0.007809,0.383954,0.118035,0.487106,N/A,N/A


In [203]:
final_result.iloc[0,[3,5,7,9]]

1 Month Return Percentile       0.91404
3 Months Return Percentile     0.604585
6 Months Return Percentile     0.925501
12 Months Return Percentile    0.905444
Name: 0, dtype: object

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [207]:

for i in final_result.index:
    final_result.loc[[i],['HQM(high-quality momentum score)']]=final_result.iloc[i,[3,5,7,9]].mean()

final_result

,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,182.734772,0.036680,0.914040,0.003677,0.604585,0.323740,0.925501,0.546590,0.905444,0.837393,N/A
1,AOS,86.984901,-0.058920,0.329513,-0.146915,0.160458,-0.211295,0.094556,-0.138192,0.146132,0.182665,N/A
2,ABT,79.692963,-0.010823,0.736390,-0.011467,0.561605,0.114884,0.687679,0.019425,0.312321,0.574499,N/A
3,ABBV,112.073029,0.016110,0.873926,-0.073457,0.372493,0.070793,0.575931,0.134964,0.504298,0.581662,N/A
4,ACN,157.273666,-0.014246,0.704871,-0.008344,0.567335,0.129978,0.719198,0.025160,0.326648,0.579513,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
344,NRG,17.250000,0.051318,0.945559,0.102917,0.851003,0.320161,0.916905,0.979596,0.982808,0.924069,N/A
345,NUE,49.777840,-0.155210,0.022923,-0.218329,0.060172,-0.277182,0.045845,-0.297880,0.042980,0.04298,N/A
346,NVDA,168.318756,0.037314,0.916905,0.039463,0.696275,0.091028,0.641834,1.485717,0.997135,0.813037,N/A
347,NVR,56.366020,-0.096151,0.094556,-0.160291,0.128940,-0.007809,0.383954,0.118035,0.487106,0.273639,N/A


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [216]:
eventual_result=final_result.sort_values(by='HQM(high-quality momentum score)',ascending=False)
Best_Momentum_stocks=eventual_result.iloc[0:50].reset_index(drop=True)
Best_Momentum_stocks

,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,GEV,74.891090,0.120240,0.991404,0.374182,0.997135,1.066927,1.000000,1.582422,1.000000,0.997135,N/A
1,AVGO,47.620228,0.338248,1.000000,0.266941,0.982808,0.345704,0.951289,1.084367,0.985673,0.979943,N/A
2,FOX,30.274151,0.061006,0.962751,0.226579,0.977077,0.399204,0.971347,0.666333,0.945559,0.964183,N/A
3,DECK,65.489998,0.026717,0.899713,0.286946,0.988539,0.384113,0.965616,0.725069,0.962751,0.954155,N/A
4,FOXA,71.075516,0.056071,0.957020,0.183136,0.951289,0.369669,0.959885,0.617552,0.934097,0.950573,N/A
5,KMI,19.357944,0.052064,0.948424,0.158779,0.919771,0.438099,0.977077,0.680618,0.951289,0.94914,N/A
6,ETR,108.750000,0.032832,0.908309,0.182195,0.948424,0.459715,0.985673,0.548516,0.908309,0.937679,N/A
7,EQT,300.170013,0.138211,0.994269,0.315346,0.991404,0.342947,0.945559,0.374116,0.799427,0.932665,N/A
8,ANET,66.693390,0.102405,0.982808,0.108196,0.859599,0.281949,0.893983,0.824370,0.974212,0.92765,N/A
9,NRG,17.250000,0.051318,0.945559,0.102917,0.851003,0.320161,0.916905,0.979596,0.982808,0.924069,N/A


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [219]:
# 建立随时输入的一个函数
def portfolio_input():
    global portfolio_size
    portfolio_size=input("How much money you have?")

    try:
        float(portfolio_size)
    except ValueError:
        print("Enter some real shit!")
        portfolio_size=input("How much money you have?")

#试一下
portfolio_input()
print(portfolio_size)

#就算每个买多少
position_size= float(portfolio_size)/len(Best_Momentum_stocks)
print(position_size)

for i in Best_Momentum_stocks.index:
    Best_Momentum_stocks.loc[i,'Number of Share to Buy']=math.floor(float(position_size)/Best_Momentum_stocks.loc[i,'Price'])

Best_Momentum_stocks

100000
2000.0


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,GEV,74.891090,0.120240,0.991404,0.374182,0.997135,1.066927,1.000000,1.582422,1.000000,0.997135,26
1,AVGO,47.620228,0.338248,1.000000,0.266941,0.982808,0.345704,0.951289,1.084367,0.985673,0.979943,41
2,FOX,30.274151,0.061006,0.962751,0.226579,0.977077,0.399204,0.971347,0.666333,0.945559,0.964183,66
3,DECK,65.489998,0.026717,0.899713,0.286946,0.988539,0.384113,0.965616,0.725069,0.962751,0.954155,30
4,FOXA,71.075516,0.056071,0.957020,0.183136,0.951289,0.369669,0.959885,0.617552,0.934097,0.950573,28
5,KMI,19.357944,0.052064,0.948424,0.158779,0.919771,0.438099,0.977077,0.680618,0.951289,0.94914,103
6,ETR,108.750000,0.032832,0.908309,0.182195,0.948424,0.459715,0.985673,0.548516,0.908309,0.937679,18
7,EQT,300.170013,0.138211,0.994269,0.315346,0.991404,0.342947,0.945559,0.374116,0.799427,0.932665,6
8,ANET,66.693390,0.102405,0.982808,0.108196,0.859599,0.281949,0.893983,0.824370,0.974212,0.92765,29
9,NRG,17.250000,0.051318,0.945559,0.102917,0.851003,0.320161,0.916905,0.979596,0.982808,0.924069,115


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [220]:
# put it as excel
Best_Momentum_stocks.to_excel('My Second Shit.xlsx',
                         sheet_name="1",
                         index=False)

## Saving Our Excel Output

As before, saving our Excel output is very easy: